In [1]:
import sys
import gzip
from itertools import islice
import pandas as pd
import rdkit
from tqdm import tqdm


# Ignore rdkit warnings: https://github.com/rdkit/rdkit/issues/2683
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [2]:
print(rdkit.__version__)

2020.03.1


In [3]:
from warnings import warn

In [4]:
from rdkit.Chem.Descriptors import NumRadicalElectrons

def iter_mols():
    """ Iterate over the SDF file, yielding a pandas Series for each molecule.
    While this is likely not the fastest way to load the data, the SDF file
    format can be read by many different softwares, and the loop allows additional
    post-processing to occur while reading the molecules.
    
    Some errors for rotational constants are expected for molecules that have
    infinite values.
    
    Here, 20200415_radical_database.sdf.gz has been downloaded from figshare to
    the same directory.
    """
    
    with gzip.open('20200415_radical_database.sdf.gz') as sdffile:
        mol_suppl = rdkit.Chem.ForwardSDMolSupplier(sdffile, removeHs=False)
        for mol in tqdm(mol_suppl, total=289639):
            props = mol.GetPropsAsDict()
            props['mol'] = mol
            for item in ['AtomCharges', 'AtomSpins', 'VibFreqs', 'IRIntensity', 'RotConstants']:
                if item in props:
                    try:
                        props[item] = eval(props[item])
                    except NameError:
                        warn("Error with molecule {} property {}".format(props['SMILES'], item))
                    
            props['type'] = 'molecule' if NumRadicalElectrons(mol) == 0 else 'fragment'
            props['Name'] = mol.GetProp('_Name')
            yield pd.Series(props)

In [5]:
# Load the molecules into a pandas dataframe.
# This takes approximately 5 minutes for my compute node. Faster read/write
# speeds can be obtained for subsequent analysis by saving as a pickle or hdf5 file.
df = pd.DataFrame(iter_mols())

  0%|          | 1/289639 [00:00<9:50:59,  8.17it/s]/projects/rlmolecule/pstjohn/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Error with molecule [OH] property RotConstants
/projects/rlmolecule/pstjohn/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Error with molecule [H] property RotConstants
100%|██████████| 289639/289639 [06:08<00:00, 786.99it/s]


In [6]:
# Here, the 'Name' field can be used to find the corresponding Gaussian logfile in the
# seperate log database as {Name}.log.gz (i.e., 1_0.log.gz)
df.head()

,SMILES,Enthalpy,FreeEnergy,SCFEnergy,AtomCharges,RotConstants,VibFreqs,IRIntensity,mol,type,Name,AtomSpins
0,NCCCC(=O)O,-362.905738,-362.946238,-363.053128,"[-0.631259, -0.180356, -0.296748, -0.340264, 0...","[4.81599609, 1.46905765, 1.23652591]","[71.352, 91.0435, 214.6419, 250.3987, 305.2394...","[1.8576, 4.0578, 7.1859, 2.7184, 1.4606, 21.03...",<rdkit.Chem.rdchem.Mol object at 0x7feb448433f0>,molecule,1_0,NaN
1,[CH2]CCC(=O)O,-306.919993,-306.960303,-307.032883,"[-0.387596, -0.200858, -0.336819, 0.349212, -0...","[8.18219608, 1.72810454, 1.43599343]","[69.3394, 103.9026, 156.4399, 183.1502, 358.57...","[0.2475, 0.3625, 0.4543, 2.6889, 3.5692, 15.68...",<rdkit.Chem.rdchem.Mol object at 0x7feb448573a0>,fragment,2_0,"[1.060148, -0.093393, 0.029549, -0.00055, 0.00..."
2,[NH2],-55.846147,-55.868889,-55.869061,"[-0.47305, 0.236525, 0.236525]","[386.52719429, 195.0089984, 129.61580373]","[1528.0504, 3395.7368, 3487.6835]","[29.7349, 4.9619, 0.0843]",<rdkit.Chem.rdchem.Mol object at 0x7feb44857710>,fragment,3_0,"[1.066346, -0.033173, -0.033173]"
3,[CH2]CC(=O)O,-267.642461,-267.679410,-267.725190,"[-0.388654, -0.258195, 0.343559, -0.320303, -0...","[8.97572677, 3.50902, 2.71517013]","[63.5843, 150.8306, 278.0981, 387.2301, 485.06...","[1.1738, 2.3951, 2.0713, 63.0873, 20.5683, 7.2...",<rdkit.Chem.rdchem.Mol object at 0x7feb44857a80>,fragment,4_0,"[1.063213, -0.090775, 0.013894, -0.000999, 0.0..."
4,[CH2]N,-95.132053,-95.159615,-95.186762,"[-0.303953, -0.457963, 0.137299, 0.137299, 0.2...","[146.59773253, 27.42065414, 23.85148104]","[443.0574, 621.7992, 676.8922, 940.6024, 1243....","[34.2869, 216.3978, 98.9106, 0.0283, 35.6255, ...",<rdkit.Chem.rdchem.Mol object at 0x7feb44857300>,fragment,5_0,"[0.91136, 0.161426, -0.033484, -0.033466, -0.0..."


In [7]:
# only closed-shell molecules
df_mol = df[df.type == 'molecule']

In [8]:
# Here, we demonstrate how the `bde` python module can be used to generate
# radicals calculated in this dataset; or to link closed-shell molecules
# with their associated radicals.

sys.path.append('..')
from bde.fragment import fragment_iterator

# Here, we just demonstrate for one molecule. This can be run over
# the whole dataset in approx. 30 minutes.
# `fragment_iterator` returns a python iterator, so we trigger the calculations
# and return the results to a pandas dataframe.
reaction_df = pd.DataFrame(fragment_iterator('NCCCC(=O)O'))

# For the resulting dataframe, it lists SMILES strings for the molecule and two
# resulting radicals, along with the bond_index (of the rdkit starting molecule),
# the bond_type (atom symbols of start and end bond), and whether breaking the bond 
# creates additional stereochemistry that might complicate BDE calculations (is_valid_stereo)
reaction_df

,molecule,bond_index,bond_type,fragment1,fragment2,is_valid_stereo
0,NCCCC(=O)O,0,C-N,[CH2]CCC(=O)O,[NH2],True
1,NCCCC(=O)O,1,C-C,[CH2]CC(=O)O,[CH2]N,True
2,NCCCC(=O)O,2,C-C,[CH2]C(=O)O,[CH2]CN,True
3,NCCCC(=O)O,3,C-C,O=[C]O,[CH2]CCN,True
4,NCCCC(=O)O,5,C-O,NCCC[C]=O,[OH],True
5,NCCCC(=O)O,6,H-N,[H],[NH]CCCC(=O)O,True
6,NCCCC(=O)O,7,H-N,[H],[NH]CCCC(=O)O,True
7,NCCCC(=O)O,8,C-H,[H],N[CH]CCC(=O)O,True
8,NCCCC(=O)O,9,C-H,[H],N[CH]CCC(=O)O,True
9,NCCCC(=O)O,10,C-H,[H],NC[CH]CC(=O)O,True


In [9]:
# Here we show how these data can be used to calculate bond dissociation energies,
# as (H(rad1) + H(rad2)) - H(mol). We multiply by 627.509 to convert from Hartree
# to kcal/mol

df_smiles_index = df.set_index('SMILES')
bdes = ((df_smiles_index.loc[reaction_df.fragment1].Enthalpy.values + 
         df_smiles_index.loc[reaction_df.fragment2].Enthalpy.values) -
        df_smiles_index.loc[reaction_df.molecule].Enthalpy.values) * 627.509
reaction_df['BDE'] = bdes
reaction_df

,molecule,bond_index,bond_type,fragment1,fragment2,is_valid_stereo,BDE
0,NCCCC(=O)O,0,C-N,[CH2]CCC(=O)O,[NH2],True,87.599001
1,NCCCC(=O)O,1,C-C,[CH2]CC(=O)O,[CH2]N,True,82.344241
2,NCCCC(=O)O,2,C-C,[CH2]C(=O)O,[CH2]CN,True,86.313863
3,NCCCC(=O)O,3,C-C,O=[C]O,[CH2]CCN,True,95.898435
4,NCCCC(=O)O,5,C-O,NCCC[C]=O,[OH],True,111.411713
5,NCCCC(=O)O,6,H-N,[H],[NH]CCCC(=O)O,True,99.732515
6,NCCCC(=O)O,7,H-N,[H],[NH]CCCC(=O)O,True,99.732515
7,NCCCC(=O)O,8,C-H,[H],N[CH]CCC(=O)O,True,93.322511
8,NCCCC(=O)O,9,C-H,[H],N[CH]CCC(=O)O,True,93.322511
9,NCCCC(=O)O,10,C-H,[H],NC[CH]CC(=O)O,True,96.539750
